In [1]:
import pandas as pd
import urllib.request
import numpy as np
import requests
import unicodedata
from PIL import Image

def strip_accents(text):
    try:
        text = unicode(text, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass

    text = unicodedata.normalize('NFD', text)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")

    return str(text)

## Essential functions

In [27]:
def sanitizar_nombre(name):
    return strip_accents((name.replace(" ", "_").replace("(","").replace(")","")).lower())

# RECIBE FILA DEL DATAFRAME
# precondición: el campo imagen no es NaN
def obtener_url_img(y):
    url = y[7].split('":"')[1]
    url2 = url.split(",")[0][:-1]
    return url2

# RECIBE FILA DEL DATAFRAME
def obtenerlink(y):
    #print(y)
    url = y[5].split('":"')[1]
    url2 = url2 = url.split(",")[0][:-1]
    return url2

def quote(str):
    return '"'+str+'"'


# EMPRESAS

In [22]:
# Read Timeglider csv
df = pd.read_csv("./timelines/linea_tiempo.csv",index_col=False)

In [23]:
print(df.columns)
df.head()

Index(['startdate', ' enddate', ' title', ' description', ' importance',
       ' link', ' icon', ' image', ' date_display', ' low threshold',
       ' high threshold', ' y_position'],
      dtype='object')


,startdate,enddate,title,description,importance,link,icon,image,date_display,low threshold,high threshold,y_position
0,1830-12-31 18:14:00,1830-12-31 18:14:00,Ledesma,<p></p>,50,"[{""url"":""https://drive.google.com/open?id=199_...",shapes/triangle_gray.png,"{""src"":""https://upload.wikimedia.org/wikipedia...",da,1,100,0
1,1838-04-19 07:51:00,1838-04-19 07:51:00,Delfino,<p></p>,50,"[{""url"":""http://drive.google.com/uc?export=vie...",shapes/triangle_gray.png,"{""src"":""https://raw.githubusercontent.com/mako...",da,1,100,0
2,1847-10-12 16:02:00,1847-10-12 16:02:00,Siemens,<p></p>,50,"[{""url"":""https://raw.githubusercontent.com/mak...",shapes/triangle_gray.png,"{""src"":""https://raw.githubusercontent.com/mako...",da,1,100,0
3,1857-08-27 14:08:00,1857-08-27 14:08:00,Ferrocarril del Oeste,<p></p>,50,"[{""url"":""https://drive.google.com/open?id=0B8E...",shapes/triangle_gray.png,NaN,da,1,100,0
4,1863-01-01 12:02:00,1863-01-01 12:02:00,Ferrocarriles Argentinos,<p></p>,50,"[{""url"":""http://drive.google.com/uc?export=vie...",shapes/triangle_gray.png,"{""src"":""https://raw.githubusercontent.com/mako...",da,1,100,0


## Manually filtering some data and fixing errors

In [24]:
print(df[' title'].unique())
alpargatas=["Fábrica Argentina de Alpargatas","Alpargatas"] #multiple, deberían ser en 1883 todos # pasa a ser Alpargatas
ferrum=["Ferrum"]
siam=["SIAM"]
pirelli=["Pirelli"]
shell = ["Shell"]
ypf = ["YPF"]
torre = ["Frigorífico Lisandro de la Torre"]
gm = ["General Motors"]
felfort = ["Felfort"]
sere=["La Serenísima"]
entel = ["Entel"]
havanna = ["Havanna"]
aa = ["Aerolíneas Argentinas"]
arcor = ["Arcor"]
pv = ["Central Hidroeléctrica Pueblo Viejo"]
renault = ["Renault Argentina"]
zanon = ["Cerámica Fasinpát (ex Zanón)", "Zanón"] #pasa a ser Zanón
casasco = ["Laboratorios Casasco"]
milkaut = ["Milkaut"]
banco_macro = ["Banco Macro"]
multiples = ["Alpargatas","Ferrum","SIAM","Pirelli","Shell","YPF","Frigorífico Lisandro de la Torre","General Motors","Felfort","La Serenísima","Entel","Havanna","Aerolíneas Argentinas","Arcor","Central Hidroeléctrica Pueblo Viejo","Renault Argentina","Zanón","Laboratorios Casasco","Milkaut","Banco Macro"]

['Ledesma' 'Delfino' 'Siemens' 'Ferrocarril del Oeste'
 'Ferrocarriles Argentinos' 'Semino' 'Laboratorios Casasco (2)'
 'Laboratorios Casasco (1)' 'CGFSA' 'Aguila Saint' 'Hesperdina'
 'Rigolleau' 'Fábrica Argentina de Alpargatas (2)'
 'Fábrica Argentina de Alpargatas (1)' 'Frigorífico Anglo' 'Bunge y Born'
 'Alpargatas (1)' 'Alpargatas (2)' 'Quilmes' 'La Matrona' 'Grimoldi'
 'Cerro Negro' 'British American Tabacco' 'Pirelli (2)' 'Pirelli (1)'
 'La Martona' 'Molinos Río de la Plata S.A.' 'La Cantábrica'
 'Molinos Río de la Plata' 'Banco Galicia' 'Swift' 'Frávega' 'Ferrum (2)'
 'Bayer Argentina S.A.' 'Ferrum (1)' 'SIAM (2)' 'SIAM (1)' 'Terrabusi'
 'Cerveza Schneider' 'Obras Sanitarias de la Nación' 'BGH' 'FORD'
 'Shell (2)' 'Shell (1)' 'Firestone' 'Goodyear' 'Kodak' 'Longvie' 'FIAT'
 'Roemmers' 'YPF (3)' 'YPF (2)' 'YPF (1)'
 'Frigorífico Lisandro de la Torre (1)'
 'Frigorífico Lisandro de la Torre (2)' 'Taragüi, Estancias Las Marías'
 'General Motors (1)' 'General Motors (2)' 'Felfort (2

In [26]:
# Sometimes this cell has to be ran twice
filtro_alpargatas = (df[' title'].str.contains(alpargatas[0])) | (df[' title'].str.contains(alpargatas[1]))
#df.loc[filtro_alpargatas,' title'] = "Alpargatas"
df[filtro_alpargatas].head()
for i in range(df[filtro_alpargatas].count()[0]):
    df.loc[filtro_alpargatas,' title'].iloc[i] = "Alpargatas ("+str(i)+")"
#df.loc[9,' title'] = "Alpargatas (2)"
#df.loc[12,' title'] = "Alpargatas (3)"
#df.loc[13,' title'] = "Alpargatas (4)"

filtro_zanon = (df[' title'].str.contains(zanon[0])) | (df[' title'].str.contains(zanon[1]))
df[filtro_zanon].head()
for i in range(df[filtro_zanon].count()[0]):
    df.loc[filtro_zanon,' title'].iloc[i] = "Zanón ("+str(i)+")"
#df.loc[118,' title'] = "Zanón (1)"
#df.loc[199,' title'] = "Zanón (2)"

## Create events based on timeglider csv (migration)

In [28]:
# Buscamos crear algo de esta pinta:
'''
var items = new vis.DataSet([
{id: 1, content: 'item 1', start: '2014-04-20'},
{id: 2, content: 'item 2', start: '2014-04-14'},
{id: 3, content: 'item 3', start: '2014-04-18'},
{id: 4, content: 'item 4', start: '2014-04-16', end: '2014-04-19'},
{id: 5, content:  createItem("item 5",'./img/swift.jpg',["https://makobot-prod.github.io/timelines/","blabla"])  , start: '2014-04-25'},
{id: 6, content: 'item 6', start: '2014-04-27', type: 'point'}
]);
'''
df[' image'] = df[' image'].fillna(0)
### PARECERÍA ESTAR ANDANDO
def crear_evento(x):
    idn = sanitizar_nombre(x[2])
    #print(idn)
    #reemplazar NaNs en image por 0!!
    path_img = quote("0")
    width = 0
    start = quote(x[0][:-9])
    name_empresa = x[2]
    
    if(multiples.count(x[2][:-4])==0):
        links = "["+ quote(obtenerlink(x)) + "]" #crear esta func
    else:
        #tengo multiples trabajos de esta empresa
        #se podría manejar arriba con un for supongo? en algun lado me tengo que guardar los múltiples
        name_empresa = x[2][:-4]
        idn = idn[:-2]
        filtro = df[' title'].str.contains(x[2][:-4])
        df_links = df[filtro]
        links = "["
        df_links = df_links.apply(obtenerlink,axis=1)
        for item in df_links.iteritems():
            url = item[1]
            links += quote(url) + ","
        links = links[:-1] + "]" #saco la , al pedo y agrego el corchete final
    if(x[7]!= 0):
        path_img = quote('./img/'+idn+'.png')
        im = Image.open('./timelines/img/'+idn+'.png')
        width, height = im.size
        width = int(width*48/height)
    
    content  = "createItem("+ quote(name_empresa) +","+ path_img +","+ str(width) +","+ links + ")"
    
    return "{id: " + quote(idn) + ", content: " + content + ", start: " + start + "}"
            
#crear_evento(df.loc[118])
#df.loc[118]
filtro = df[' title'].str.contains("Banco Macro")
df[filtro]

,startdate,enddate,title,description,importance,link,icon,image,date_display,low threshold,high threshold,y_position
130,1978-09-04 11:53:00,1978-09-04 11:53:00,Banco Macro (1),<p></p>,50,"[{""url"":""https://drive.google.com/open?id=10Zg...",shapes/triangle_gray.png,"{""src"":""https://s3.amazonaws.com/timeglider_us...",da,1,100,0
131,1978-09-04 23:57:00,1978-09-04 23:57:00,Banco Macro (2),<p></p>,50,"[{""url"":""https://drive.google.com/open?id=165G...",shapes/triangle_gray.png,0,da,1,100,0


#### Apply to dataframe

In [29]:
start_line = "var items = new vis.DataSet(["
end_line = "]);"

df_sin_repes = df.copy()
filtro_repes = (df[' title'].str.contains("\(")) & (~(df[' title'].str.contains("\(1\)")))
df_sin_repes = df_sin_repes[~filtro_repes]

f = open("./timelines/empresas/trabajos_appendable.js", "a")
#f.write(start_line)
f.write("\n")

print("Procesando base de datos...")
cosas = df_sin_repes.apply(crear_evento,axis=1)
print("Guardando en archivo...")
for linea in cosas[:-1]:
    #lleno con todos los eventos hasta el anteultimo
    f.write(linea + ",\n")

#La ULTIMA fila debe ir SIN COMA en la base finalizada, y CON COMA en la appendable!
f.close()
print("Diferenciando versiones appendable y finalizada")
!cp "./timelines/empresas/trabajos_appendable.js" "./timelines/trabajos.js"

f2 = open("./timelines/empresas/trabajos.js","a")
#aca va sin coma y con linea finalizadora
f2.write(cosas.iloc[-1]+"\n")
f2.write(end_line)
f2.close()
print("Version finalizada creada")

#La version appendable va con coma y sin linea finalizadora
f = open("./timelines/empresas/trabajos_appendable.js", "a")
f.write(cosas.iloc[-1]+",")
f.close()
print("Versión appendable creada")
print("Finalizado!")

Procesando base de datos...
Guardando en archivo...
Diferenciando versiones appendable y finalizada
Version finalizada creada
Versión appendable creada
Finalizado!


## Image resizer (OVERWRITES IMAGES)

#### OBSOLETE: images are not that heavy and look better without resizing

In [35]:
#!convert "./timelines/empresas/img/*.png" -filter Lanczos -resize x48 -sharpen 0x.5 -set filename:base "%[basename]" "./timelines/empresas/img/%[filename:base].png"

Decoded /tmp/magick-3689dVLyjnH2RKVf. Dimensions: 560 x 210  (with alpha). Format: lossless. Now saving...
Saved file /tmp/magick-3689r4TsGY9Aa3Al
convert-im6.q16: improper image header `./timelines/img/coto.png' @ error/png.c/ReadPNGImage/3954.
convert-im6.q16: improper image header `./timelines/img/sheraton.png' @ error/png.c/ReadPNGImage/3954.


In [ ]:
#Delete _1 from filename for images that end in _1
#!for file in `find . -name '*_1.png'` ; do mv "$file" "${file%_1.png}.png"; done

## Timeline event images downloader

In [ ]:
#urllib.request.urlretrieve("http://www.digimouth.com/news/media/2011/09/google-logo.jpg", "local-filename.jpg")
# Quiero hacer una función que, dada una fila, me baje la imagen
df_img=df.copy()
df_img.dropna(subset=[' image'], inplace=True) 
falladas=[]
assert(False)
#path = "./img_originales/"
def image_download(x):
    empresa=sanitizar_nombre(x[2])
    # donde x es una serie de Pandas
    url = x[7].split('":"')[1]
    url2 = url.split(",")[0][:-1]
    print(url2)
    try:
        #response = requests.get(url2)
        print("Downloading img for",empresa)
        urllib.request.urlretrieve(url2, path + empresa.lower() + ".png")
    except Exception as exc:
        print('[!!!] {err}'.format(err=exc),"retrieving img for", empresa ,"aborted")
        falladas.append(empresa)

for index, row in df_img.iterrows():
    print(index)
    image_download(df_img.loc[index])


In [99]:
print(falladas)

['cgfsa', 'correo_oca', 'don_satur', 'sheraton', 'arcos_dorados_s.a.', 'telecentro']


In [18]:
if(not np.isnan(df.loc[1][7].isnull()):
    print("Not a nan")
    print(df.loc[1][7])
    urllib.request.urlretrieve(df.loc[1][7], path+df.loc[1][2].lower()+".jpg")

# HISTORIC EVENTS TIMELINES

In [7]:
df = pd.read_csv("./timelines/1unidad/1unidad.csv",index_col=False)
df.columns = df.columns.str.replace(' ', '')

In [13]:
df.tail()

,startdate,enddate,title,description,importance,link,icon,image,date_display,lowthreshold,highthreshold,y_position
115,1919-01-19 21:02:00,1919-01-19 21:02:00,7 al 14-01-1919: la Semana Trágica (cont.),<p></p>,65,"[{""url"":""https://drive.google.com/file/d/1dPoj...",shapes/triangle_gray.png,"{""src"":""http://www.cooperacion2005.es/wp-conte...",da,1,100,0
116,1922-06-03 21:02:00,1922-06-03 21:02:00,03-06-1922: Creación de la Dirección General d...,<p></p>,60,"[{""url"":""https://drive.google.com/file/d/1Imhr...",shapes/triangle_gray.png,"{""src"":""http://www.cooperacion2005.es/wp-conte...",da,1,100,0
117,1929-10-24 21:02:00,1929-10-24 21:02:00,24-10-1929: Crack del 29 y Gran Depresión de l...,<p></p>,65,"[{""url"":""http://www.cooperacion2005.es/wp-cont...",shapes/triangle_gray.png,"{""src"":""http://www.cooperacion2005.es/wp-conte...",da,1,100,0
118,1929-10-29 21:02:00,1929-10-29 21:02:00,29-10-1929: Crack del 29 y Gran Depresión (bis),<p></p>,65,"[{""url"":""https://drive.google.com/file/d/1CL0t...",shapes/triangle_gray.png,"{""src"":""http://www.cooperacion2005.es/wp-conte...",da,1,100,0
119,4760-01-02 12:00:00,4760-01-02 12:00:00,476 Fin del Imperio Romano de Occidente y de l...,<p></p>,60,"[{""url"":""http://www.cooperacion2005.es/wp-cont...",shapes/triangle_gray.png,"{""src"":""http://www.cooperacion2005.es/wp-conte...",ye,1,100,0


## Migrate events from Timeglider

In [ ]:
def crear_evento(x):
    idn = sanitizar_nombre(x[2])
    #print(idn)
    #reemplazar NaNs en image por 0!!
    path_img = quote("0")
    width = 0
    start = quote(x[0][:-9])
    name_empresa = x[2]
    
    if(multiples.count(x[2][:-4])==0):
        links = "["+ quote(obtenerlink(x)) + "]" #crear esta func
    else:
        #tengo multiples trabajos de esta empresa
        #se podría manejar arriba con un for supongo? en algun lado me tengo que guardar los múltiples
        name_empresa = x[2][:-4]
        idn = idn[:-2]
        filtro = df[' title'].str.contains(x[2][:-4])
        df_links = df[filtro]
        links = "["
        df_links = df_links.apply(obtenerlink,axis=1)
        for item in df_links.iteritems():
            url = item[1]
            links += quote(url) + ","
        links = links[:-1] + "]" #saco la , al pedo y agrego el corchete final
    if(x[7]!= 0):
        path_img = quote('./img/'+idn+'.png')
        im = Image.open('./timelines/img/'+idn+'.png')
        width, height = im.size
        width = int(width*48/height)
    
    content  = "createItem("+ quote(name_empresa) +","+ path_img +","+ str(width) +","+ links + ")"
    
    return "{id: " + quote(idn) + ", content: " + content + ", start: " + start + "}"

### Apply to dataframe

In [ ]:
start_line = "var items = new vis.DataSet(["
end_line = "]);"

df_sin_repes = df.copy()
filtro_repes = (df[' title'].str.contains("\(")) & (~(df[' title'].str.contains("\(1\)")))
df_sin_repes = df_sin_repes[~filtro_repes]

f = open("./timelines/empresas/trabajos_appendable.js", "a")
#f.write(start_line)
f.write("\n")

print("Procesando base de datos...")
cosas = df_sin_repes.apply(crear_evento,axis=1)
print("Guardando en archivo...")
for linea in cosas[:-1]:
    #lleno con todos los eventos hasta el anteultimo
    f.write(linea + ",\n")

#La ULTIMA fila debe ir SIN COMA en la base finalizada, y CON COMA en la appendable!
f.close()
print("Diferenciando versiones appendable y finalizada")
!cp "./timelines/empresas/trabajos_appendable.js" "./timelines/trabajos.js"

f2 = open("./timelines/empresas/trabajos.js","a")
#aca va sin coma y con linea finalizadora
f2.write(cosas.iloc[-1]+"\n")
f2.write(end_line)
f2.close()
print("Version finalizada creada")

#La version appendable va con coma y sin linea finalizadora
f = open("./timelines/empresas/trabajos_appendable.js", "a")
f.write(cosas.iloc[-1]+",")
f.close()
print("Versión appendable creada")
print("Finalizado!")